## 데이터 인코딩

머신러닝 알고리즘은 문자열 데이터 속성을 입력받지 않기 때문에 모든 데이터를 숫자형으로 표현해야 한다. 이렇게 문자형 같은 사람이 이해할 수 있는 카테고리 속성을 컴퓨터가 이해할 수 있는 숫자값으로 변환하는 것을 인코딩한다고 말한다. 주로 2가지 방법이 있다.
- 레이블(Label) 인코딩
- 원-핫(One-Hot) 인코딩

### 1. 레이블 인코딩
- 상품 분류가 TV, 냉장고, 전자레인지, ... 와 같다면 이를 0, 1, 2, ... 와 같이 숫자로 인코딩하는 방식
- 이때 의도하지는 않았지만 1보다 2가 큰 값을 가지게 되는 단점이 있다

In [9]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

encoder = LabelEncoder()
# fit -> transform의 순서. 이때 fit_transform으로 한번에 처리할 수도 있다
encoder.fit(items)
# fit을 하면 encoder.classes_ 에 items의 고유한 컬럼만 들어간다
labels = encoder.transform(items)
print("인코딩 변환값:", labels)

인코딩 변환값: [0 1 4 5 3 3 2 2]


In [10]:
print("인코딩 클래스:", encoder.classes_)
print("디코딩 원본값:", encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']
디코딩 원본값: ['전자레인지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


### 2. 원핫인코딩
- TV, 냉장고, 전자레인지, ... 처럼 피처 고유 값의 개수만큼 컬럼을 생성해서 해당하는 컬럼에만 1을 표시하고 나머지 컬럼에는 0을 표시하는 인코딩 방식
- TV, 냉장고, 전자렌지, ... 같은 컬럼이 생성되고 TV인 경우 TV 컬럼에만 1이, 나머지 컬럼에는 0이 들어간다
- OneHotEncoder 클래스를 사용하거나 pandas의 get_dummies를 사용하면 된다

OneHotEncoder를 사용해서 인코딩하기

In [17]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# items 리스트를 2차원 ndarray로 변환한다
items = np.array(items).reshape(-1, 1)

# One-Hot 인코딩을 적용한다 (fit -> transform의 순서. 이때 fit_transform으로 한번에 처리할 수도 있다)
oh_encoder = OneHotEncoder()
oh_encoder.fit(items)
oh_labels = oh_encoder.transform(items)

# OneHotEncoder로 변환한 결과는 희소행렬(Sparse Matrix)이므로 toarray()를 이용해서 밀집행렬(Dense Matrix)로 변환
print(oh_labels)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 3)	1.0
  (5, 3)	1.0
  (6, 2)	1.0
  (7, 2)	1.0


In [18]:
print(oh_labels.toarray())
print(oh_labels.shape)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
(8, 6)


pandas의 get_dummies 활용해서 인코딩하기

In [22]:
import pandas as pd

df = pd.DataFrame({'item': ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']})
# dtype=int로 설정하면 0, 1값을 그렇지 않으면 True, False 값을 반환한다
pd.get_dummies(df, dtype=int)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## 피처 스케일링과 정규화

- 목적: 스케일링을 해주면 성능이 향상될 가능성이 있는 모델들이 있음. Tree 같은 모델은 스케일링의 영향을 별로 받지 않고, 선형 계열의 Logistic Regression, SVM 같은 모델은 영향을 받을 가능성이 있는 편이다.
- 표준화(Standardization)
    - 피처 각각을 평균이 0이고 분산이 1인 표준 정규 분포(standard normal distribution)를 가진 값으로 변환하는 것
    - 참고로 정규 분포(normal distribution)는 가우시안 분포라고도 불리며 수집된 데이터들이 평균 근처에 모여있어 종형곡선(bell curve) 형태로 그려진다. 이 중 평균이 0이고 분산이 1인 것이 표준 정규 분포이다
    - 사이킷런의 StandardScaler 사용
- 정규화(Normalization)
    - 데이터 값을 0과 1사이의 범위 값으로 변환 (음수 값이 있으면 -1 ~ 1 값으로 변환)
    - 사이킷런의 MinMaxScaler 사용

### StandardScaler

In [29]:
from sklearn.datasets import load_iris
import pandas as pd

# 먼저 iris 데이터셋을 불러와서 각 피처의 평균값과 분산값을 살펴본다
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

print("스케일링 전 iris 데이터 피처별 평균 값")
print(iris_df.mean())
print("\n스케일링 전 iris 데이터 피처별 분산 값")
print(iris_df.var())

스케일링 전 iris 데이터 피처별 평균 값
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

스케일링 전 iris 데이터 피처별 분산 값
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [37]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()

# fit -> transform의 단계로 진행 (fit을 하면 스케일링에 필요한 평균, 표준편차 값들을 구하는 듯)
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform을 하면 ndarray를 반환하는데 이를 df로 아래와 같이 변환해줌
iris_scaled_df = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print("스케일링 후 iris 데이터 피처별 평균 값")
print(iris_scaled_df.mean())
print("\n스케일링 후 iris 데이터 피처별 분산 값")
print(iris_scaled_df.var())

# 결과가 평균 0, 분산/표준편차가 1에 근사한 값으로 반환되는 것을 확인할 수 있다

스케일링 후 iris 데이터 피처별 평균 값
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

스케일링 후 iris 데이터 피처별 분산 값
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


### MinMaxScaler

In [38]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 생성
scaler = MinMaxScaler()

# 동일하게 fit -> transform의 과정을 수행한다
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

iris_scaled_df = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print("스케일링 후 iris 데이터 피처별 최소값")
print(iris_scaled_df.min())
print("\n스케일링 후 iris 데이터 피처별 최대값")
print(iris_scaled_df.max())

스케일링 후 iris 데이터 피처별 최소값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

스케일링 후 iris 데이터 피처별 최대값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


### 학습/테스트 데이터에 Scaler 적용 시 유의사항

In [40]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터를 0 ~ 10, 테스트 데이터에 0 ~ 5를 부여함
# scaler의 fit, transform은 2차원 이상 데이터만 가능하므로 reshape 수행
train_array = np.arange(0, 11).reshape(-1, 1)
test_array = np.arange(0, 6).reshape(-1, 1)

In [47]:
# MinMaxScaler를 적용해보자
scaler = MinMaxScaler()

scaler.fit(train_array)
train_scaled = scaler.transform(train_array)

print("원본 데이터", train_array.reshape(-1))
print("스케일링 후 데이터", train_scaled.reshape(-1))
# 스케일링 후 원본데이터의 값 10이 1로 스케일링 된 것을 확인할 수 있음

원본 데이터 [ 0  1  2  3  4  5  6  7  8  9 10]
스케일링 후 데이터 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [49]:
# 이때 테스트 데이터를 가지고 다시 fit -> transform을 해보면
scaler.fit(test_array)
test_scaled = scaler.transform(test_array)

print("원본데이터", test_array.reshape(-1))
print("스케일링 후 데이터", test_scaled.reshape(-1))
# 스케일링 후 원본데이터 값 5가 1로 스케일링 된 것을 확인할 수 있음
# 즉, 서로 다른 데이터로 scaler를 만들어서 10과 5가 모두 1로 스케일링 되어버림

원본데이터 [0 1 2 3 4 5]
스케일링 후 데이터 [0.  0.2 0.4 0.6 0.8 1. ]


In [53]:
# 이를 방지하기 위해서는 모든 데이터로 스케일러를 만들고 스케일링을 적용하든
# 학습 데이터로 생성한 스케일러로 테스트 데이터에 transform만 수행해야 한다
# 그래야 동일한 기준으로 스케일링을 진행할 수 있다
scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)

print("학습 데이터 스케일링 후", train_scaled.reshape(-1))

# 여기서 fit을 호출하지 않고 동일한 scaler를 활용해 transform을 수행한다
test_scaled = scaler.transform(test_array)
print("테스트 데이터 스케일링 후", test_scaled.reshape(-1))

# 여기서는 학습 및 테스트 데이터셋 모두에서 5 -> 0.5로 스케일링 된 것을 확인할 수 있다

학습 데이터 스케일링 후 [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
테스트 데이터 스케일링 후 [0.  0.1 0.2 0.3 0.4 0.5]
